<a href="https://colab.research.google.com/github/CagataySencan/TrafficSignRecognition/blob/main/gtsdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verisetini Kaggle'dan Çekmek İçin Gerekli İşlemler


In [ ]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle

In [6]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
! kaggle datasets download safabouguezzi/german-traffic-sign-detection-benchmark-gtsdb

100% 1.61G/1.61G [00:13<00:00, 132MB/s]
100% 1.61G/1.61G [00:13<00:00, 126MB/s]


In [ ]:
! unzip german-traffic-sign-detection-benchmark-gtsdb.zip

## YOLO FORMATTING

TXT VE JPGLERİN OLUŞTURULMASI

In [10]:
import pandas as pd
import os
import cv2

In [12]:
# Her fotoğraf için annotation txt dosyalarının oluşturulması için ön hazırlık
ann = pd.read_csv('/content/gt.txt',
                  names=['ImageID',
                         'XMin',
                         'YMin',
                         'XMax',
                         'YMax',
                         'ClassID'],
                  sep=';')

# Adding new empty columns to dataFrame to save numbers for YOLO format
ann['center x'] = ''
ann['center y'] = ''
ann['width'] = ''
ann['height'] = ''

# Calculating bounding box's center in x and y for all rows
# Saving results to appropriate columns
ann['center x'] = (ann['XMax'] + ann['XMin']) / 2
ann['center y'] = (ann['YMax'] + ann['YMin']) / 2

# Calculating bounding box's width and height for all rows
# Saving results to appropriate columns
ann['width'] = ann['XMax'] - ann['XMin']
ann['height'] = ann['YMax'] - ann['YMin']

# Getting Pandas dataFrame that has only needed columns
# By using 'loc' method we locate here all rows but only specified columns
# By using copy() we create separate dataFrame not just a reference to the previous one and, in this way, initial dataFrame will not be changed
r = ann.loc[:, ['ImageID',
                'ClassID',
                'center x',
                'center y',
                'width',
                'height']].copy()

# Check point
# Showing first 5 rows from the dataFrame
print(r.head())

     ImageID  ClassID  center x  center y  width  height
0  00000.ppm       11     794.5     428.5     41      35
1  00001.ppm       40    1003.5     410.0     41      44
2  00001.ppm       38     414.0     523.0     56      58
3  00001.ppm       13    1002.0     362.5     58      55
4  00002.ppm       39     949.0     534.0    114     116


In [13]:
'''Normalizing YOLO numbers according to the real image width and height
Saving annotations in txt files
Converting images from ppm to jpg'''

def annotation(fullPathtoDataset):
  os.chdir(fullPathtoDataset)

  for current_dir, dirs, files in os.walk('.'):
     for f in files:
        if f.endswith('.ppm'):
          image_ppm = cv2.imread(f)
          h, w = image_ppm.shape[:2]

          # Slicing only name of the file without extension
          image_name = f[:-4]

          sub_r = r.loc[r['ImageID'] == f].copy()

          # Normalizing calculated bounding boxes' coordinates according to the real image width and height
          sub_r['center x'] = sub_r['center x'] / w
          sub_r['center y'] = sub_r['center y'] / h
          sub_r['width'] = sub_r['width'] / w
          sub_r['height'] = sub_r['height'] / h

          resulted_frame = sub_r.loc[:, ['ClassID',
                                         'center x',
                                         'center y',
                                         'width',
                                         'height']].copy()

          # Checking if there is no any annotations for current image
          if resulted_frame.isnull().values.all():
              continue

          # Preparing path where to save txt file
          path_to_save = fullPathtoDataset + '/' + image_name + '.txt'

          # Saving resulted Pandas dataFrame into txt file
          resulted_frame.to_csv(path_to_save, header=False, index=False, sep=' ')

          # Preparing path where to save jpg image
          path_to_save = fullPathtoDataset + '/' + image_name + '.jpg'

          # Saving image in jpg format by OpenCV function that uses extension to choose format to save with
          cv2.imwrite(path_to_save, image_ppm)

In [18]:
annotation('/content/TestIJCNN2013/TestIJCNN2013Download')

In [25]:
from pathlib import Path

rootdir = '/content/TrainIJCNN2013/TrainIJCNN2013'
for path in Path(rootdir).iterdir():
  if path.is_dir():
    path=str(path)
    annotation(path)

TRAİN VE TEST TXTLERİN OLUŞTURULMASI

In [35]:
os.chdir('/content/TestIJCNN2013/TestIJCNN2013Download')
p = []
for current_dir, dirs, files in os.walk('.'):
  # Going through all files
  for f in files:
    # Checking if filename ends with '.jpg'
    if f.endswith('.jpg'):
      path_to_save_into_txt_files = '/content/TestIJCNN2013/TestIJCNN2013Download' + '/' + f

      # Appending the line into the list
      p.append(path_to_save_into_txt_files + '\n')

# Creating file train.txt and writing 85% of lines in it  
os.chdir('/content/TestIJCNN2013')
with open('test.txt', 'w') as test_txt:
  # Going through all elements of the list
  for e in p:
    # Writing current path at the end of the file
    test_txt.write(e)

In [36]:
p = []

rootdir = '/content/TrainIJCNN2013/TrainIJCNN2013'
for path in Path(rootdir).iterdir():
  if path.is_dir():
    path=str(path)
    os.chdir(path)
    for current_dir, dirs, files in os.walk('.'):
      # Going through all files
      for f in files:  
        # Checking if filename ends with '.jpg'
        if f.endswith('.jpg'): 
          path_to_save_into_txt_files = path + '/' + f

          # Appending the line into the list
          p.append(path_to_save_into_txt_files + '\n')

# Creating file train.txt and writing 85% of lines in it  
os.chdir('/content/TrainIJCNN2013')
with open('train.txt', 'w') as train_txt:
  # Going through all elements of the list
  for e in p:
    # Writing current path at the end of the file
    train_txt.write(e)

CLASSES.NAMES VE DATA.DATA

In [37]:
classes = { 0:'Hız Limiti (20km/s)',
            1:'Hız Limiti (30km/s)', 
            2:'Hız Limiti (50km/s)', 
            3:'Hız Limiti (60km/s)', 
            4:'Hız Limiti (70km/s)', 
            5:'Hız Limiti (80km/s)', 
            6:'Hız Limitinin bitişi (80km/s)', 
            7:'Hız Limiti (100km/s)', 
            8:'Hız Limiti (120km/s)', 
            9:'Geçiş yok', 
            10:'Ağırlığı 3.5 ton üstü olan araçlar geçemez', 
            11:'Kavşakta geçiş hakkı', 
            12:'Öncelikli yol', 
            13:'Yol ver', 
            14:'Dur', 
            15:'Araç giremez', 
            16:'3.5 Ton üzeri araçlar yasak', 
            17:'Giriş yok', 
            18:'Genel uyarı', 
            19:'Sola tehlikeli viraj', 
            20:'Sağa tehlikeli viraj', 
            21:'Sağa tehlikeli devamlı virajlar', 
            22:'Engebeli yol', 
            23:'Kaygan yol', 
            24:'Sağdan daralan yol', 
            25:'Yol çalışması', 
            26:'Trafik işaretleri', 
            27:'Yaya geçidi', 
            28:'Okul geçidi', 
            29:'Bisiklet yolu', 
            30:'Buzlanmaya dikkat',
            31:'Vahşi hayvan çıkabilir', 
            32:'Hız sınırı sonu', 
            33:'Sadece sağa dönüş var', 
            34:'Sadece sola dönüş var', 
            35:'Dönüş yok', 
            36:'Sola dönüş yok', 
            37:'Sağa dönüş yok', 
            38:'Sağdan gidin', 
            39:'Soldan gidin', 
            40:'Dönüş önceliği', 
            41:'Geçiş olmayan yolun sonu', 
            42:'3.5 Ton üzeri araçlar girebilir' }

c = pd.DataFrame([classes])
c.to_csv('/content/classes.names', header=False, index=False, sep='\n')

In [38]:
# CUSTOM_DATA.DATA FILE

with open('/content/ts_data.data', 'w+') as out:
  out.write('classes = 43\n')
  out.write('train = /content/TrainIJCNN2013/train.txt\n')
  out.write('valid = /content/TestIJCNN2013/test.txt\n')
  out.write('names = /content/classes.names\n')
  out.write('backup = backup')

out.close()

# Darknet


In [ ]:
!cd '/content'
!git clone https://github.com/AlexeyAB/darknet/
!cd '/darknet'
!make

In [ ]:
!make